Importation des library

In [312]:
import pandas as pd 
import json
import os
import yaml
from pathlib import Path
from typing import Optional, Union, Dict, Any, List
import re

### EXTRACTION

In [313]:
def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Goal:
        Function to load configuration parameters from a YAML file.
    Parameters:
        config_filename (str): Relative or absolute path to the YAML file.
    Returns:
        dict: A dictionary containing the configuration parameters.
    """

    config_path = Path(config_filename)

    # Si le chemin est relatif, on le résout depuis la racine du projet
    if not config_path.is_absolute():
        project_root = Path(__file__).resolve().parents[1]
        config_path = project_root / config_path

    if not config_path.exists():
        raise FileNotFoundError(f"Config file not found: {config_path}")

    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config

def fct_read_csv(root_file: str) -> pd.DataFrame:
    """
    Goal:
        Function to read a CSV file and return a pandas DataFrame.
    Parameters:
        root_file (str): Relative or absolute path to the CSV file.
    Returns:
        pd.DataFrame: The DataFrame containing the data from the CSV file.
    """
    seps = [',', ';', '|', '\t']

    file_path = Path(root_file)

    if not file_path.is_absolute():
        try:
            # Cas script Python
            project_root = Path(__file__).resolve().parents[1]
        except NameError:
            # Cas notebook Jupyter
            project_root = Path.cwd().parent

        file_path = project_root / file_path

    if not file_path.exists():
        print(f"Erreur : fichier {file_path} introuvable")
        return pd.DataFrame()

    for sep in seps:
        try:
            df = pd.read_csv(
                file_path,
                sep=sep,
                encoding="utf-8",
                skipinitialspace=True
            )

            # Si plus d'une colonne → bon séparateur
            if df.shape[1] > 1:
                return df

        except Exception:
            continue

    print(f"Aucun séparateur valide trouvé pour {file_path}")
    return pd.DataFrame()


In [314]:
# Load configuration parameters from config.yaml
config_path = os.path.join(Path.cwd().parent, 'config.yaml')
config = fct_load_config(config_path)


root_csv_2014 = config['root_csv_2014']
 

# read data in dfs
df_2014 = fct_read_csv(root_csv_2014)



### TRANSFORMATION

Exploration du dataframe

In [315]:
df_2014.head()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,2014,12 Jun 2014 - 17:00,Group A,Arena de Sao Paulo,Sao Paulo,Brazil,3,1,Croatia,NaN,62103.0,1,1,NISHIMURA Yuichi (JPN),SAGARA Toru (JPN),NAGI Toshiyuki (JPN),255931,300186456,BRA,CRO
1,2014,13 Jun 2014 - 13:00,Group A,Estadio das Dunas,Natal,Mexico,1,0,Cameroon,NaN,39216.0,0,0,ROLDAN Wilmar (COL),CLAVIJO Humberto (COL),DIAZ Eduardo (COL),255931,300186492,MEX,CMR
2,2014,13 Jun 2014 - 16:00,Group B,Arena Fonte Nova,Salvador,Spain,1,5,Netherlands,NaN,48173.0,1,1,Nicola RIZZOLI (ITA),Renato FAVERANI (ITA),Andrea STEFANI (ITA),255931,300186510,ESP,NED
3,2014,13 Jun 2014 - 18:00,Group B,Arena Pantanal,Cuiaba,Chile,3,1,Australia,NaN,40275.0,2,1,Noumandiez DOUE (CIV),YEO Songuifolo (CIV),BIRUMUSHAHU Jean Claude (BDI),255931,300186473,CHI,AUS
4,2014,14 Jun 2014 - 13:00,Group C,Estadio Mineirao,Belo Horizonte,Colombia,3,0,Greece,NaN,57174.0,1,0,GEIGER Mark (USA),HURD Sean (USA),FLETCHER Joe (CAN),255931,300186471,COL,GRE


In [316]:
df_2014.columns

Index(['Year', 'Datetime', 'Stage', 'Stadium', 'City', 'Home Team Name',
       'Home Team Goals', 'Away Team Goals', 'Away Team Name',
       'Win conditions', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Referee', 'Assistant 1', 'Assistant 2',
       'RoundID', 'MatchID', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')

### Transformation sur les noms de colonnes
- Sélection des colonnes pertinentes
- Normalisation des noms de colonnes
- Renommage des colonnes pour uniformisation

Sélection des colonnes pertinentes

In [317]:
# Sélection des colonnes pertinentes
df_2014_news = df_2014[['Year', 'Datetime', 'Stage', 'City', 'Home Team Name','Home Team Goals', 'Away Team Goals', 'Away Team Name']]

Normalisation des noms de colonnes

In [318]:
# Normalisation des noms de colonnes
df_2014_news.columns = df_2014_news.columns.str.lower().str.replace(" ", "_")

Renommage des colonnes pour uniformisation

In [319]:
# Renommage des colonnes pour uniformisation
news_columns = {
        "year": "edition",
        "home_team_name": "home_team",
        "away_team_name": "away_team",
        "home_team_goals": "home_result",
        "away_team_goals": "away_result",
    }

df_2014_news = df_2014_news.rename(
    columns=news_columns
)

### Transformation sur la colonne Datetime
- Création de la fonction ``normalize_datetime`` pour standardiser les dates
- Création d’une colonne ``date`` normalisée

Fonction normalize_datetime pour standardiser les dates

In [320]:
def normalize_datetime(x: Union[str, pd.Timestamp]) -> Optional[str]:
    """
    Convertit une date/heure en chaîne de caractères au format YYYYMMDDhhmmss.

    Paramètres
    ----------
    x : str | pandas.Timestamp
        Date/heure à convertir (ex: "12 Jun 2014 - 17:00").

    Retour
    ------
    str | None
        Date/heure normalisée au format YYYYMMDDhhmmss,
        ou None si la conversion échoue ou si la valeur est manquante.
    """
    try:
        # Conversion en datetime pandas (gestion automatique de plusieurs formats)
        dt = pd.to_datetime(x, dayfirst=True)

        # Formatage final en YYYYMMDDhhmmss
        return dt.strftime("%Y%m%d%H%M%S")

    except Exception:
        # Retourne None si la conversion échoue
        return None


Création d’une colonne date normalisée

In [321]:
# Création d’une colonne date normalisée
df_2014_news["date"] = df_2014_news["datetime"].apply(normalize_datetime)

### Transformation sur la colonne stage
- Récupérer le valeurs distinctes de la colonne ``stage``
- Création d'un dictionnaire ``stage_mapping`` pour convertir les noms des phases du tournoi en format uniforme et lisible.
- Normalisation des noms de la colonne ``stage``

Récupérer le valeurs distinctes de la colonne stage

In [322]:
# Récupérer le valeurs distinctes de la colonne stage
df_2014_news["stage"].unique()

array(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F',
       'Group G', 'Group H', 'Round of 16', 'Quarter-finals',
       'Semi-finals', 'Play-off for third place', 'Final'], dtype=object)

Création du dictionnaire et normalisation des noms de la colonne stage

In [323]:
# création d'un dictionnaire ``stage_mapping``
stage_mapping = {
    'Group A':'group_a',
    'Group B':'group_b',
    'Group C':'group_c',
    'Group D':'group_d',
    'Group E':'group_e',
    'Group F':'group_f',
    'Group G':'group_g',
    'Group H':'group_h',
    'Round of 16':'round_of_16',
    'Quarter-finals':'quarter_finals',
    'Semi-finals':'semi_finals',
    'Play-off for third place':'third_place',
    'Final': 'final'
}
# normalisation des noms de la colonne stage
df_2014_news["stage"] = df_2014_news["stage"].map(stage_mapping).fillna(df_2014_news["stage"])


### Test et Transformation sur la colonne ``home_team`` et ``away_team``
- Création de la fonction **test_country_column** qui détecte les anomalies dans les colonnes ``home_team`` et ``away_team``:
    - Première lettre non majuscule (not_capitalized)
    - Espaces superflus au début, à la fin ou multiples (extra_spaces)
    - Caractères spéciaux ou guillemets indésirables (special_chars)
- Test et récupération des anomalies:
    - Application **test_country_column** sur les colonnes ``home_team`` et ``away_team``
    - Stockage des résultats des anomalies dans un dictionnaire
- Correction des valeurs problématiques par un mapping ou d’une fonction de nettoyage qui :
    - Supprime les caractères indésirables
    - Corrige les majuscules
    - Supprime les espaces superflus

In [324]:
def test_country_column(df: pd.DataFrame, column: str) -> Dict[str, List[str]]:
    """
    Teste la colonne pour :
    - majuscule initiale
    - espaces superflus (début, fin, multiples)
    - caractères spéciaux ou accentués
    - guillemets indésirables dans la chaîne
    """
    issues = {
        "not_capitalized": [],
        "special_chars": [],
        "extra_spaces": []
    }

    for val in df[column].dropna().unique():
        val_str = str(val)
        val_strip = val_str.strip()
        
        # Majuscule initiale
        if not val_strip[0].isupper():
            issues["not_capitalized"].append(val_str)
        
        # Espaces en début ou fin
        if val_str != val_strip:
            issues["extra_spaces"].append(val_str)
        
        # Espaces multiples à l'intérieur
        if "  " in val_str:
            if val_str not in issues["extra_spaces"]:
                issues["extra_spaces"].append(val_str)
        
        # Caractères spéciaux ou accents incorrects (garde lettres accentuées)
        if re.search(r'[^A-Za-zÀ-ÖØ-öø-ÿ\s\-]', val_strip):
            if val_str not in issues["special_chars"]:
                issues["special_chars"].append(val_str)
        
        # Détection spécifique des guillemets " ou '
        if '"' in val_str or "'" in val_str:
            if val_str not in issues["special_chars"]:
                issues["special_chars"].append(val_str)

    return issues

Détection des anomalies et correction des valeurs dans la colonne ``home_team``

In [325]:
# Détection des anomalies dans la colonne home_team
home_team_results = test_country_column(df_2014_news, "home_team")
print(home_team_results)

{'not_capitalized': ['rn">Bosnia and Herzegovina'], 'special_chars': ["C�te d'Ivoire", 'rn">Bosnia and Herzegovina'], 'extra_spaces': []}


Correction des valeurs de la colonnne ``home_team`` par un mapping

In [326]:
home_team_mapping = {
    'rn">Bosnia and Herzegovina':'Bosnia and Herzegovina',
    "C�te d'Ivoire":'Ivory Coast'
}
# normalisation des noms de la colonne stage
df_2014_news["home_team"] = df_2014_news["home_team"].map(home_team_mapping).fillna(df_2014_news["home_team"])
df_2014_news["home_team"].unique()


array(['Brazil', 'Mexico', 'Spain', 'Chile', 'Colombia', 'Uruguay',
       'England', 'Ivory Coast', 'Switzerland', 'France', 'Argentina',
       'Germany', 'IR Iran', 'Ghana', 'Belgium', 'Russia', 'Australia',
       'Cameroon', 'Japan', 'Italy', 'Honduras', 'Nigeria',
       'Korea Republic', 'USA', 'Netherlands', 'Croatia', 'Costa Rica',
       'Greece', 'Bosnia and Herzegovina', 'Ecuador', 'Portugal',
       'Algeria'], dtype=object)

Test final

In [327]:
home_team_mapping = test_country_column(df_2014_news, "home_team")
print(home_team_mapping)

{'not_capitalized': [], 'special_chars': [], 'extra_spaces': []}


Détection des anomalies et correction des valeurs dans la colonne ``away_team``

In [328]:
away_team_results = test_country_column(df_2014_news, "away_team")
print(away_team_results)

{'not_capitalized': ['rn">Bosnia and Herzegovina'], 'special_chars': ['rn">Bosnia and Herzegovina', "C�te d'Ivoire"], 'extra_spaces': []}


Correction des valeurs de la colonnne ``home_team`` par un mapping

In [329]:
home_team_mapping = {
    'rn">Bosnia and Herzegovina':'Bosnia and Herzegovina',
    "C�te d'Ivoire":'Ivory Coast'
}
# normalisation des noms de la colonne stage
df_2014_news["away_team"] = df_2014_news["away_team"].map(home_team_mapping).fillna(df_2014_news["away_team"])
df_2014_news["away_team"].unique()

array(['Croatia', 'Cameroon', 'Netherlands', 'Australia', 'Greece',
       'Costa Rica', 'Italy', 'Japan', 'Ecuador', 'Honduras',
       'Bosnia and Herzegovina', 'Portugal', 'Nigeria', 'USA', 'Algeria',
       'Mexico', 'Korea Republic', 'Chile', 'Ivory Coast', 'England',
       'France', 'IR Iran', 'Ghana', 'Russia', 'Spain', 'Brazil',
       'Uruguay', 'Colombia', 'Argentina', 'Switzerland', 'Germany',
       'Belgium'], dtype=object)

Test final

In [330]:
results = test_country_column(df_2014_news, "away_team")
print(results)

{'not_capitalized': [], 'special_chars': [], 'extra_spaces': []}


### Tri chronologique (date) et attribution d’un ID incrémental (colonne match_id)
- Trie le dataframe par la colonne ``date`` afin que les matchs les plus anciens apparaissent en premier
- Création d'une colonne ``match_id`` avec un identifiant unique et incrémental correspondant à l’ordre chronologique

In [331]:
# Trier par date (ascendant : plus ancien en premier)
df_2014_news.sort_values("date", inplace=True)
# Réinitialiser l'index incrémental pour match_id
df_2014_news["match_id"] = range(1, len(df_2014_news) + 1)

### Réorganisation des colonnes du DataFrame

In [332]:
# Réorganisation des colonnes du DataFrame
df_2014_news = df_2014_news[["match_id", "date", "home_team", "away_team", "home_result", "away_result", "stage", "edition", "city"]]

### DataFrame traité et sauvegardé en CSV

In [333]:
project_root = Path.cwd().parent

csv_file_path = os.path.join(project_root, "data/processed_WorldCupMatches2014.csv")

df_2014_news.to_csv(csv_file_path, sep=";", index=False)